<a href="https://colab.research.google.com/github/priyal-khapra/Projects/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing the dependencies

In [1]:
import numpy as np
import pandas as pd
import difflib
# finding the closest match of the movie provided by the user
from sklearn.feature_extraction.text import TfidfVectorizer
# used to convert textual data into numerical value
from sklearn.metrics.pairwise import cosine_similarity
# which movie has the highest similarity score

data collection and preprocessing

In [2]:
movies_data = pd.read_csv('/content/movies.csv')

In [3]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies_data.shape

(4803, 24)

In [5]:
# selecting the relevant features for recommendation
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [6]:
# replacing the missing values/ null values with null string
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [7]:
# combining all the 5 selected features 
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [8]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [9]:
# converting the text data to feature vectors
vectorizer = TfidfVectorizer()
features_vectors = vectorizer.fit_transform(combined_features)

In [10]:
print(features_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

cosine similarity

In [11]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(features_vectors)

In [12]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [13]:
similarity.shape
# so every single movie in the dataset is compared with every other movies in the data set and then is assigned a similarity score

(4803, 4803)

In [14]:
# getting the movie name from the user
movie_name = input('Enter your favourite movie name:')

Enter your favourite movie name:Hangover


In [15]:
# creating a list with all the movie names given in the dataset 
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [16]:
# finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name , list_of_all_titles)
print(find_close_match)

['The Hangover', 'Tango', 'Rango']


In [17]:
close_match = find_close_match[0]

In [18]:
# finding the index of the movie with title 
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

1356


In [19]:
# getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.03845947043916149), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.017874318473410422), (15, 0.0), (16, 0.024261094490921314), (17, 0.0), (18, 0.0038900756428820577), (19, 0.030271047044930468), (20, 0.0), (21, 0.0), (22, 0.03152643163021836), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.04099235420670399), (38, 0.0), (39, 0.0), (40, 0.008628036159331984), (41, 0.0), (42, 0.004068421595953236), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.004283396418924043), (56, 0.029127575834722395), (57, 0.004528099342435968), (58, 0.004610003002036298), (59, 0.0), (60, 0.0), (61, 0.0), (62, 0.0), (63, 0.0), (64, 0.0), (65, 0.0), (66, 0.030789017498074785), (67, 0.0), (68, 0.0), (69, 0.0), (70

In [20]:
len(similarity_score)

4803

In [21]:
# sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score , key = lambda x:x[1] , reverse = True)
print(sorted_similar_movies)

[(1356, 0.9999999999999999), (427, 0.46562455570517175), (923, 0.18585226779917904), (3056, 0.16208875775569906), (641, 0.16151286262613132), (1712, 0.14811147719428783), (2356, 0.13229137300932775), (2699, 0.11973105935937986), (4696, 0.11813235227742824), (1934, 0.11600677507295182), (1321, 0.11205271806126119), (2747, 0.10072228033303793), (3186, 0.09665513399462874), (3425, 0.09189384341712227), (1310, 0.09076704236183065), (623, 0.08908485826994787), (1055, 0.08827613578877647), (1340, 0.08818567421617897), (1572, 0.08707091891094279), (778, 0.08690108095584058), (321, 0.08637988318774487), (1358, 0.0861585129535576), (3951, 0.0849682520869387), (1348, 0.08438452081004343), (2132, 0.08293663125519277), (817, 0.08289883651591294), (871, 0.08236597511103438), (2071, 0.0821809336504047), (2334, 0.08207693630292505), (187, 0.08198634309832005), (738, 0.08190677564886055), (2940, 0.08142587607604455), (829, 0.08107938874083472), (3275, 0.0796616866758266), (3276, 0.0796292822917296), (

In [22]:
# print the name of similar movies based on the index
print('Movies suggested for you: \n')
i = 1 
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, ".", title_from_index)
    i += 1

Movies suggested for you: 

1 . The Hangover
2 . The Hangover Part II
3 . Failure to Launch
4 . Cedar Rapids
5 . Due Date
6 . Last Vegas
7 . Youth in Revolt
8 . Fun Size
9 . Weekend
10 . Say It Isn't So
11 . Texas Rangers
12 . The Big Hit
13 . Trust the Man
14 . Go
15 . Paul Blart: Mall Cop 2
16 . Dinner for Schmucks
17 . The Muppets
18 . We're the Millers
19 . Forgetting Sarah Marshall
20 . Meet Dave
21 . The Campaign
22 . Austin Powers: The Spy Who Shagged Me
23 . Tim and Eric's Billion Dollar Movie
24 . Aloha
25 . Vampires Suck
26 . American Wedding
27 . Gigli
28 . Silver Linings Playbook
29 . Birdman
